In [37]:
import numpy as np
import pandas as pd
import pmdarima as pm
import matplotlib.pyplot as plt

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
historical_data = pd.read_csv('../../data/raw/historical_rent_by_suburb.csv')

historical_data.drop(columns=['Unnamed: 1'], inplace=True)
historical_data.rename(columns={'Unnamed: 0': 'Suburb'}, inplace=True)
historical_data.set_index('Suburb', inplace=True)

all_columns = [col for col in historical_data.columns if "." in col]
historical_data = historical_data[all_columns]

historical_data.rename(columns=lambda x: x.split(".")[0], inplace=True)
historical_data = historical_data.iloc[1:, :]

historical_data = historical_data.reset_index()
historical_data.loc[historical_data.index[158:], "Suburb"] = "All Suburbs"

historical_data

,Suburb,Mar-00,Jun-00,Sep-00,Dec-00,Mar-01,Jun-01,Sep-01,Dec-01,Mar-02,...,Dec-22,Mar-23,Jun-23,Sep-23,Dec-23,Mar-24,Jun-24,Sep-24,Dec-24,Mar-25
0,Albert Park-Middle Park-West St Kilda,$260,$260,$270,$275,$275,$280,$280,$290,$300,...,$525,$545,$555,$600,$600,$660,$675,$693,$700,$700
1,Armadale,$200,$200,$205,$210,$215,$220,$225,$230,$233,...,$460,$490,$500,$525,$560,$560,$590,$600,$600,$625
2,Carlton North,$260,$260,$265,$270,$270,$275,$280,$280,$290,...,$600,$620,$630,$650,$670,$680,$690,$680,$700,$720
3,Carlton-Parkville,$251,$260,$260,$260,$260,$260,$260,$270,$270,...,$450,$500,$530,$550,$550,$570,$580,$585,$600,$600
4,CBD-St Kilda Rd,$320,$320,$320,$320,$320,$320,$320,$320,$320,...,$480,$550,$580,$600,$620,$645,$650,$650,$650,$650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Wanagaratta,$125,$125,$130,$130,$130,$130,$130,$135,$135,...,$380,$380,$390,$390,$395,$400,$400,$420,$420,$411
155,Warragul,$130,$135,$135,$135,$135,$135,$140,$140,$150,...,$430,$440,$450,$450,$460,$473,$480,$490,$500,$520
156,Warrnambool,$130,$135,$135,$135,$140,$140,$140,$145,$145,...,$420,$420,$430,$450,$460,$460,$475,$480,$480,$500
157,Wodonga,$145,$145,$150,$150,$150,$150,$150,$150,$150,...,$410,$410,$420,$420,$430,$440,$450,$450,$460,$475


In [ ]:
def forecast_suburb(data):
    plot = None
    return plot

In [ ]:
results = {}
forecasts = {}

for suburb in historical_data['Suburb'].unique():
    # Extract suburb data
    suburb_data = historical_data[historical_data['Suburb'] == suburb].drop(columns=['Suburb']).T
    suburb_data.index = pd.to_datetime(suburb_data.index, format='%b-%y')
    suburb_data.index = suburb_data.index.to_period("Q")
    suburb_data.columns = [suburb]

    ts = suburb_data[suburb].astype(float)
    
    # Fit auto_arima
    model = pm.auto_arima(
        ts,
        seasonal=True,        # Quarterly data, so seasonality makes sense
        m=4,                  # 4 quarters in a year
        trace=True,           # Prints progress
        error_action='ignore',
        suppress_warnings=True
    )
    
    # Store the fitted model
    results[suburb] = model
    
    # Forecast next 20 quarters (5 years)
    forecast = model.predict(n_periods=20)
    forecast_index = pd.period_range(ts.index[-1] + 1, periods=20, freq='Q')
    forecast_series = pd.Series(forecast, index=forecast_index)
    
    forecasts[suburb] = forecast_series
    
    # Plot historical + forecast
    plt.figure(figsize=(10, 5))
    plt.plot(ts.index.to_timestamp(), ts, label='Historical')
    plt.plot(forecast_series.index.to_timestamp(), forecast_series, label='Forecast', linestyle='--')
    plt.title(f"{suburb} Forecast for Next 5 Years")
    plt.xlabel("Date")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    
    # Save figure
    plt.savefig(f"{suburb}_forecast.png")
    plt.close()  # Close to avoid displaying immediately

       Albert Park-Middle Park-West St Kilda
2000Q1                                 $260 
2000Q2                                 $260 
2000Q3                                 $270 
2000Q4                                 $275 
2001Q1                                 $275 
...                                      ...
2024Q1                                 $660 
2024Q2                                 $675 
2024Q3                                 $693 
2024Q4                                 $700 
2025Q1                                 $700 

[101 rows x 1 columns]
